In [ ]:
import spotipy
import spotipy.util as util
import pandas as pd # for later
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
cid = 'aa402d49d83b447aa9d473332fc8413b'
secret = '051e674e3aeb40b1b1781f0efb31a0d5'

In [ ]:
#Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
playlist_creator = "spotify"
playlist_id = "37i9dQZF1DX5IDTimEWoTd"

In [ ]:
playlist = sp.user_playlist_tracks(playlist_creator, playlist_id)["items"]

In [ ]:
playlist

[{'added_at': '2022-05-13T16:03:31Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
   'href': 'https://api.spotify.com/v1/users/',
   'id': '',
   'type': 'user',
   'uri': 'spotify:user:'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'single',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3gIRvgZssIb9aiirIg0nI3'},
      'href': 'https://api.spotify.com/v1/artists/3gIRvgZssIb9aiirIg0nI3',
      'id': '3gIRvgZssIb9aiirIg0nI3',
      'name': 'Jeremy Zucker',
      'type': 'artist',
      'uri': 'spotify:artist:3gIRvgZssIb9aiirIg0nI3'},
     {'external_urls': {'spotify': 'https://open.spotify.com/artist/5JMLG56F1X5mFmWNmS0iAp'},
      'href': 'https://api.spotify.com/v1/artists/5JMLG56F1X5mFmWNmS0iAp',
      'id': '5JMLG56F1X5mFmWNmS0iAp',
      'name': 'Chelsea Cutler',
      'type': 'artist',
      'uri': 'spotify:artist:5JMLG56F1X5mFmWNmS0iAp'}],
    'available_markets': [],
    'ex

In [ ]:
playlist[1]["track"]["popularity"]

76

In [ ]:
playlist[0]["track"]["album"]["images"][0]["url"]

'https://i.scdn.co/image/ab67616d0000b273dccb079c492cf99e02e3255b'

In [ ]:
playlist[1]["track"]["album"]["images"]

[{'height': 640,
  'url': 'https://i.scdn.co/image/ab67616d0000b27389aff4625958eac8d16535c7',
  'width': 640},
 {'height': 300,
  'url': 'https://i.scdn.co/image/ab67616d00001e0289aff4625958eac8d16535c7',
  'width': 300},
 {'height': 64,
  'url': 'https://i.scdn.co/image/ab67616d0000485189aff4625958eac8d16535c7',
  'width': 64}]

In [ ]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist", "album", "track_name",  "popularity","img_url","track_id", 
                             "danceability", "energy", "key", "loudness", "mode", "speechiness",
                             "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Create empty dict
    playlist_features = {}
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["popularity"] = track["track"]["popularity"]
        try:
            playlist_features["img_url"]= track["track"]["album"]["images"][0]["url"]
        except IndexError:
            playlist_features["img_url"] = 'null'
    
            
        
        playlist_features["track_id"] = track["track"]["id"]
        
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[6:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [ ]:
playlist_df = analyze_playlist(playlist_creator, playlist_id)

In [ ]:
playlist_df

,artist,album,track_name,popularity,img_url,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Jeremy Zucker,this is how you fall in love,this is how you fall in love,0,https://i.scdn.co/image/ab67616d0000b273dccb07...,0742PcMPo2Akmo4vHC29VZ,0.52,0.204,7,-12.488,1,0.0355,0.000025,0.126,0.274,79.682,174587,4
1,Ed Sheeran,The Joker And The Queen (feat. Taylor Swift),The Joker And The Queen (feat. Taylor Swift),76,https://i.scdn.co/image/ab67616d0000b27389aff4...,6N1K5OVVCopBjGViHs2IvP,0.528,0.309,0,-7.398,1,0.0329,0.000002,0.282,0.307,133.83,185422,4
2,Forest Blakk,Fall Into Me,Fall Into Me,67,https://i.scdn.co/image/ab67616d0000b273942b53...,7Lv78wiIJcKgND9Zi7AIOt,0.45,0.483,2,-6.754,1,0.0572,0,0.127,0.206,118.305,226067,4
3,MAX,Butterflies (feat. FLETCHER),Butterflies (feat. FLETCHER),61,https://i.scdn.co/image/ab67616d0000b273613b08...,3hJmzuKkiCtLDTANdB52Hm,0.686,0.618,11,-4.177,1,0.0375,0,0.15,0.485,88.096,191250,4
4,One Direction,FOUR (Deluxe),Night Changes,87,https://i.scdn.co/image/ab67616d0000b273d304ba...,5O2P9iiztwhomNh8xkR9lJ,0.672,0.52,8,-7.747,1,0.0353,0,0.115,0.37,120.001,226600,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Charlie Brennan,Little Things,Little Things,0,https://i.scdn.co/image/ab67616d0000b273f83c15...,74mU4zNiHJl990F6gqBG4p,0.593,0.488,4,-9.72,0,0.154,0,0.351,0.423,164.183,146837,4
96,John Mayer,XO,XO,70,https://i.scdn.co/image/ab67616d0000b273924db3...,7cpCU3Denug5NGZsSpQl8v,0.431,0.377,2,-9.753,1,0.0288,0,0.211,0.353,173.67,213627,4
97,Matt Hartke,London's Song,London's Song,63,https://i.scdn.co/image/ab67616d0000b2732bf11e...,1Mk9ou2m6hsEZJYuNRJFpO,0.562,0.266,4,-14.852,1,0.0409,0.000626,0.0983,0.493,88.968,178573,4
98,Tom Walker,Better Half of Me,Better Half of Me,56,https://i.scdn.co/image/ab67616d0000b2731c3368...,114xcQz8sZ6fxRBTZmgmNE,0.576,0.453,3,-4.4,1,0.0294,0,0.267,0.418,107.89,195452,4


In [ ]:
playlist_dict = {
    "1" : ("Vivek Amin", "3Ne6bYetQeQ4uxxY9HOP9z"), 
    "2" : ("Uday", "3qF07OWQ5nfBJnHLOu0YyD"),
    "3" : ("pateln0901", "3rCoVttxG66PdVwhpYyrOK"),
    "4" : ("Sharu John" , "1ArALRGx8MePcZvuQLw1UR"),
     "5" : ("1163103644" , "01pjvKcZ2B10Mj6E1S4dmt"),
    "indian pop" : ("Filtr India" , "5KRBZT2yL1PclwlhEYiMzM"),
   
     "indian hip-hop" : ("Filter India" , "2gU7fwyjOOKwzgTVGE5wNV"),
     "bollywood old" : ("Sunny Patel" , "4HF4e71bGTMWbJXNInd0vA") ,
      "bollywood 2018" : ("Musico Playlists" , "37mQGNwL1EtzH9Kh49UCG0") ,
      "bollywood 2015-17" : ("Marian Chan" , "4bbaZRbhWR0rA6K1Cs2hwU") ,
    "bollywood 2014" : ("Sharu John" , "5UOILvvQrX3NqLsGkfIloT") ,
    "bollywood 2013" : ("Arth" , "0d864oKOYuHkkQ5SNuTjYy") ,
    "bollywood 2012" : ("M.TAHIR21" , "4l5HfnvKvPBgDEijROPzTX") ,
    "bollywood 2011" : ("arti23oomar" , "0riz8TsHRfhZpCc1DwG3Ct") ,
    "bollywood 1995-2000" : ("Vivek Khare" , "03lGhTwp3vpjbJtijG3pqe") ,
       "bollywood famous" : ("Spotify" , "37i9dQZF1DWZNJXX2UeBij") ,
    "bollywood famous 2000-10" : ("Music Styles" , "2Kj5NUtVetggUDHPIGC9U7") 
    
    
    
    
    
    
    
    
    
}

In [ ]:
playlist_dict.items()

dict_items([('1', ('Vivek Amin', '3Ne6bYetQeQ4uxxY9HOP9z')), ('2', ('Uday', '3qF07OWQ5nfBJnHLOu0YyD')), ('3', ('pateln0901', '3rCoVttxG66PdVwhpYyrOK')), ('4', ('Sharu John', '1ArALRGx8MePcZvuQLw1UR')), ('5', ('1163103644', '01pjvKcZ2B10Mj6E1S4dmt')), ('indian pop', ('Filtr India', '5KRBZT2yL1PclwlhEYiMzM')), ('indian hip-hop', ('Filter India', '2gU7fwyjOOKwzgTVGE5wNV')), ('bollywood old', ('Sunny Patel', '4HF4e71bGTMWbJXNInd0vA')), ('bollywood 2018', ('Musico Playlists', '37mQGNwL1EtzH9Kh49UCG0')), ('bollywood 2015-17', ('Marian Chan', '4bbaZRbhWR0rA6K1Cs2hwU')), ('bollywood 2014', ('Sharu John', '5UOILvvQrX3NqLsGkfIloT')), ('bollywood 2013', ('Arth', '0d864oKOYuHkkQ5SNuTjYy')), ('bollywood 2012', ('M.TAHIR21', '4l5HfnvKvPBgDEijROPzTX')), ('bollywood 2011', ('arti23oomar', '0riz8TsHRfhZpCc1DwG3Ct')), ('bollywood 1995-2000', ('Vivek Khare', '03lGhTwp3vpjbJtijG3pqe')), ('bollywood famous', ('Spotify', '37i9dQZF1DWZNJXX2UeBij')), ('bollywood famous 2000-10', ('Music Styles', '2Kj5NUtVetgg

In [ ]:
def analyze_playlist_dict(playlist_dict):
    
    # Loop through every playlist in the dict and analyze it
    for i, (key, val) in enumerate(playlist_dict.items()):
        playlist_df = analyze_playlist(*val)
        # Add a playlist column so that we can see which playlist a track belongs too
        playlist_df["playlist"] = key
        # Create or concat df
        if i == 0:
            playlist_dict_df = playlist_df
        else:
            playlist_dict_df = pd.concat([playlist_dict_df, playlist_df], ignore_index = True)
            
    return playlist_dict_df

In [ ]:

multiple_playlist_df = analyze_playlist_dict(playlist_dict)

In [ ]:
multiple_playlist_df

,artist,album,track_name,popularity,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,playlist
0,Sanjay Leela Bhansali,Bajirao Mastani (Original Motion Picture Sound...,"Aaj Ibaadat (From ""Bajirao Mastani"")",5,5Gn6UwbHRq8VurLB1EFlAa,0.492,0.628,3,-5.032,1,0.0302,0.000002,0.0954,0.211,111.718,284453,1,1
1,Shankar-Ehsaan-Loy,Wake Up Sid (Original Motion Picture Soundtrack),Aaj Kal Zindagi,52,4MnDLNhqL6HuSzFO4b4rvJ,0.639,0.617,4,-8.897,1,0.0329,0.00558,0.099,0.405,134.95,254507,4,1
2,Shankar-Ehsaan-Loy,Best of SEL,"Aaj Kal Zindagi (From ""Wake Up Sid "")",32,3uhMb1xevkSqoU8re2lVOQ,0.639,0.617,4,-8.897,1,0.0329,0.00558,0.099,0.405,134.95,254507,4,1
3,Various Artists,I Love You...Forever,"Aankhon Mein Neendein (From ""We Are Family"")",8,4YyScV41EbT8KFM1cU6xzj,0.485,0.696,7,-7.546,1,0.0598,0.000018,0.117,0.605,93.704,302467,4,1
4,Pritam,Barfi! (Original Motion Picture Soundtrack),Aashiyan,56,7ttlemwytO21npSmLKqTBg,0.829,0.622,10,-7.293,1,0.0518,0.000003,0.167,0.775,131.971,236373,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1227,Vishal Bhardwaj,Kaminey,Dhan Te Nan,54,2BOY77LvMTUOfLcnfFAJGl,0.766,0.809,4,-6.206,1,0.161,0.000166,0.0796,0.803,91.987,279934,4,bollywood famous 2000-10
1228,Rajesh Roshan,Krrish,Pyaar Ki Ek Kahani,52,6mqTUI7ucRJrVvoXLrDNRe,0.698,0.844,6,-4.661,1,0.0645,0.000054,0.0707,0.82,79.973,388707,4,bollywood famous 2000-10
1229,Rajesh Roshan,Krrish,Dil Na Diya,51,2PaAG9RNH3HKL57LKc3GJI,0.733,0.933,7,-4.583,1,0.105,0.0691,0.0615,0.696,132.992,354253,4,bollywood famous 2000-10
1230,Various Artists,Bluff Master,Right Here Right Now,50,3OGQ5gp3dkZ38eIjRKLF4V,0.854,0.449,10,-10.762,1,0.0939,0.015,0.111,0.533,93.995,182533,4,bollywood famous 2000-10


In [ ]:
multiple_playlist_df.to_csv('Bollywood_all_songs.csv')

In [ ]:
multiple_playlist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232 entries, 0 to 1231
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   artist            1232 non-null   object
 1   album             1232 non-null   object
 2   track_name        1232 non-null   object
 3   track_id          1232 non-null   object
 4   danceability      1232 non-null   object
 5   energy            1232 non-null   object
 6   key               1232 non-null   object
 7   loudness          1232 non-null   object
 8   mode              1232 non-null   object
 9   speechiness       1232 non-null   object
 10  instrumentalness  1232 non-null   object
 11  liveness          1232 non-null   object
 12  valence           1232 non-null   object
 13  tempo             1232 non-null   object
 14  duration_ms       1232 non-null   object
 15  time_signature    1232 non-null   object
 16  playlist          1232 non-null   object
dtypes: object(17)


In [ ]:
df = multiple_playlist_df.sort_values("track_name", ascending=False)

In [ ]:
df

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,playlist
363,KASYAP,khoya sa,khoya sa,2yspDGJntwkUjvzjSbeqZg,0.767,0.617,11,-7.881,1,0.0517,0.00002,0.255,0.714,89.977,194880,4,indian pop
971,Rashid Khan,Raaz 3,Zindagi Se,3leLzzwwZEaJYGO05B01ku,0.571,0.787,11,-7.044,0,0.118,0,0.117,0.365,154.057,290749,4,bollywood 2012
816,Pritam,Bajrangi Bhaijaan,Zindagi Kuch Toh Bata (Reprise),6TGX01ExmdHyOG72fqUU3Q,0.528,0.615,5,-7.118,1,0.0372,0.000111,0.104,0.225,123.988,258879,4,bollywood 2015-17
814,Pritam,Bajrangi Bhaijaan,Zindagi Kuch Toh Bata,62VUSRdvSo53fRsfveton5,0.52,0.484,7,-8.399,1,0.0327,0.00143,0.15,0.421,124.136,262420,4,bollywood 2015-17
1183,Rajesh Roshan,Kites,Zindagi Do Pal Ki,3s6H6UW7yPu2rsF8CGki60,0.678,0.771,6,-2.849,1,0.0266,0.00253,0.149,0.462,105.976,253200,4,bollywood famous 2000-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,Why Don't We,8 Letters,8 Letters,6AkFxZzBfyzV5RF7mGkASM,0.607,0.478,1,-5.702,0,0.031,0,0.0928,0.255,88.979,190027,4,bollywood 2018
483,Naezy,302,302,5Ve6MrmOKSGrrZTjEwf99r,0.645,0.785,9,-3.7,1,0.255,0,0.135,0.657,105.111,132000,4,indian hip-hop
488,Raga,2006,2006,67rj9DKodDfNXQQx3f5BaZ,0.748,0.409,3,-10.86,0,0.863,0,0.11,0.577,169.779,176777,4,indian hip-hop
519,,,,3m3lYYV5wFbgvpASdTtaK2,0.711,0.77,5,-7.133,0,0.132,0,0.103,0.5,89.091,231650,4,indian hip-hop


In [ ]:
df.drop_duplicates(keep='first', inplace=True)
df.sort_values("track_name", ascending=False)

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,playlist
363,KASYAP,khoya sa,khoya sa,2yspDGJntwkUjvzjSbeqZg,0.767,0.617,11,-7.881,1,0.0517,0.00002,0.255,0.714,89.977,194880,4,indian pop
971,Rashid Khan,Raaz 3,Zindagi Se,3leLzzwwZEaJYGO05B01ku,0.571,0.787,11,-7.044,0,0.118,0,0.117,0.365,154.057,290749,4,bollywood 2012
816,Pritam,Bajrangi Bhaijaan,Zindagi Kuch Toh Bata (Reprise),6TGX01ExmdHyOG72fqUU3Q,0.528,0.615,5,-7.118,1,0.0372,0.000111,0.104,0.225,123.988,258879,4,bollywood 2015-17
814,Pritam,Bajrangi Bhaijaan,Zindagi Kuch Toh Bata,62VUSRdvSo53fRsfveton5,0.52,0.484,7,-8.399,1,0.0327,0.00143,0.15,0.421,124.136,262420,4,bollywood 2015-17
1183,Rajesh Roshan,Kites,Zindagi Do Pal Ki,3s6H6UW7yPu2rsF8CGki60,0.678,0.771,6,-2.849,1,0.0266,0.00253,0.149,0.462,105.976,253200,4,bollywood famous 2000-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
698,Why Don't We,8 Letters,8 Letters,6AkFxZzBfyzV5RF7mGkASM,0.607,0.478,1,-5.702,0,0.031,0,0.0928,0.255,88.979,190027,4,bollywood 2018
483,Naezy,302,302,5Ve6MrmOKSGrrZTjEwf99r,0.645,0.785,9,-3.7,1,0.255,0,0.135,0.657,105.111,132000,4,indian hip-hop
488,Raga,2006,2006,67rj9DKodDfNXQQx3f5BaZ,0.748,0.409,3,-10.86,0,0.863,0,0.11,0.577,169.779,176777,4,indian hip-hop
519,,,,3m3lYYV5wFbgvpASdTtaK2,0.711,0.77,5,-7.133,0,0.132,0,0.103,0.5,89.091,231650,4,indian hip-hop
